In [1]:
from deepmusicmoduleloader import ModuleLoader
from deepmusic.keyboardcell import keyboardcell
import deepmusic.songstruct as music
import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'deepmusicmoduleloader'

In [ ]:
def build_network(self):
    inout_dim = ModuleLoader.batch_builders.get_module().get_input_dim()
    
    #note data
    with tf.name_scope('placeholder_input'):
        self.inputs = [
            tf.placeholder(tf.float32, 
                           [self.args.batch_size, input_dim],
                          name = 'input')
        ]
        
    # target key pressed, bin classification
    with tf.name_scope('placeholder_targets'):
        self.target = [
            tf.placeholder(tf.int32,
                          [self.batch_size],
                          name ='target')
        ]
        
    #Hidden
    with tf.name_scope('placehoplder_use_prev'):
        self.use_prev = [
            tf.placeholder(tf.bool,
                          name = 'use_prev')
        ]
        
    #define our netwoek
    self.loop_processing = ModuleLoader.loop_processings.build_module(self.args)
    def loop_rnn(prev, i):
        next_input = self.loop_processing(prev)
        return tf.cond(self.prev[i], lambda: next_input, lambda: self.inputs[i])
    
    #build seq2seq model
    self.outputs, self.final_state = tf.nn.seq2seq.rnn_decoder(
        decoder_inputs = self.inputs,
        initial_state = None,
        cell = KeyboardCell,
        loop_function = loop_rnn
    )
    
    #train step
    #lets define loss funct
    
    loss_fct = tf.nn.seq2seq.sequence_loss(
        self.outputs,
        self.target,
        softmax_loss_function = tf.nn.softmax.cross_entropy_with_logits,
        average_across_timesteps = True,
        average_across_batch = True)
    
    #initilize the optimizer
    opt = tf.train.AdamOptimizer(
        learning_rate = self.curent_learning_rate,
        beta1 = 0.9,
        beta2 = 0.999,
        epsilon = 1e-08)
    
    self.opt_op = opt.minimize(loss_fct)